# Neptune Result Fetching Notebook

In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3

# required to enable non-blocking interaction:
%gui qt5

# import ipywidgets as widgets
# widgets.Widget.control_comm_live = False

import sys
from copy import deepcopy
from typing import Dict, List, Tuple, Optional, Callable, Union, Any
from typing_extensions import TypeAlias
from neuropy.utils.result_context import IdentifyingContext
from nptyping import NDArray
import neuropy.utils.type_aliases as types
from collections import defaultdict

import numpy as np
import pandas as pd
import re
from pathlib import Path
from datetime import datetime

from neuropy.utils.indexing_helpers import PandasHelpers
from pyphocorehelpers.indexing_helpers import partition_df
# Set the maximum number of columns to display
pd.set_option('display.max_columns', 100)

import IPython
from pyphocorehelpers.programming_helpers import IPythonHelpers
from pyphocorehelpers.notebook_helpers import NotebookCellExecutionLogger
from pyphocorehelpers.assertion_helpers import Assert

# Jupyter-lab enable printing for any line on its own (instead of just the last one in the cell)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# _notebook_path:Path = Path(IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())).resolve() # Finds the path of THIS notebook

# Plotting
# import pylustrator # customization of figures
import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
_bak_rcParams = mpl.rcParams.copy()

matplotlib.use('Qt5Agg')

# Switch to the desired interactivity mode
plt.interactive(True)

import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
from pyphocorehelpers.plotting.media_output_helpers import fig_to_clipboard
from pyphocorehelpers.Filesystem.path_helpers import file_uri_from_path, sanitize_filename_for_Windows
from pyphocorehelpers.gui.Jupyter.simple_widgets import fullwidth_path_widget, simple_path_display_widget
from pyphoplacecellanalysis.Pho2D.plotly.Extensions.plotly_helpers import plotly_helper_save_figures, _helper_build_figure, plotly_pre_post_delta_scatter, plot_across_sessions_scatter_results
from pyphocorehelpers.assertion_helpers import Assert

# from ..PendingNotebookCode import plot_across_sessions_scatter_results, plot_histograms, plot_stacked_histograms
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import find_csv_files, find_HDF5_files, find_most_recent_files
from pyphoplacecellanalysis.Pho2D.statistics_plotting_helpers import plot_histograms_across_sessions, plot_histograms, plot_stacked_histograms

from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.MultiContextComputationFunctions.DirectionalPlacefieldGlobalComputationFunctions import DecoderDecodedEpochsResult
from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import load_across_sessions_exported_files, _process_and_load_exported_file, _common_cleanup_operations

from pyphocorehelpers.programming_helpers import metadata_attributes
from pyphocorehelpers.function_helpers import function_attributes

from pyphoplacecellanalysis.SpecificResults.AcrossSessionResults import build_session_t_delta, _new_process_csv_files, _old_process_csv_files

debug_print: bool = False

_TODAY_DAY_ONLY_DATE: str = "2024-09-24"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_Apogee"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_GL"
TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_Lab"
# TODAY_DAY_DATE: str = f"{_TODAY_DAY_ONLY_DATE}_rMBP"

print(f'TODAY_DAY_DATE: {TODAY_DAY_DATE}')

types.session_str: TypeAlias = str # a unique session identifier

import neptune # for logging progress and results
from neptune.types import File
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import Neptuner, AutoValueConvertingNeptuneRun, set_environment_variables, SessionDescriptorString, RunID, NeptuneRunCollectedResults
from pyphoplacecellanalysis.General.Batch.NeptuneAiHelpers import KnownNeptuneProjects

## Gets the notebook filepath for Neptune:
import IPython
from pyphocorehelpers.programming_helpers import IPythonHelpers
# notebook_filepath: str = IPythonHelpers.try_find_notebook_filepath(IPython.extract_module_locals())
# assert Path(notebook_filepath).resolve().exists(), f"found notebook filepath: '{notebook_filepath}' does not exist"
# notebook_filepath
        
neptune_kwargs = KnownNeptuneProjects.get_PhoDibaBatchProcessing_neptune_kwargs()
# neptune_kwargs = KnownNeptuneProjects.get_PhoDibaLongShortUpdated_neptune_kwargs()
neptuner = Neptuner(project_name=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])
# neptuner
# neptuner.project

Automatic pdb calling has been turned OFF
TODAY_DAY_DATE: 2024-09-24_Lab
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaBatchProcessing/


In [2]:
neptune_run_collected_results: NeptuneRunCollectedResults = neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-09-24', n_recent_results=3)
# neptune_run_collected_results: NeptuneRunCollectedResults = neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-09-20', n_recent_results=3)
# neptune_run_collected_results: NeptuneRunCollectedResults = neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-09-20', n_recent_results=3)
neptune_run_collected_results
# runs_dict, most_recent_runs_table_df = neptuner.get_most_recent_session_runs(oldest_included_run_date='2024-06-01', n_recent_results=2)
# most_recent_runs_table_df

Fetching table...: 0 [00:00, ?/s]

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaBatchProcessing/e/PHDBATCH-1380
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaBatchProcessing/e/PHDBATCH-1395
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaBatchProcessing/e/PHDBATCH-1394
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaBatchProcessing/e/PHDBATCH-1393
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaBatchProcessing/e/PHDBATCH-1392
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaBatchProcessing/e/PHDBATCH-1390
[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/commander.pho/PhoDibaBatchProcessing/e/PHDBATCH-1391
[neptune] [info   ] Neptune initialized. Open in the app: http

In [ ]:
context_indexed_runs_list_dict: Dict[IdentifyingContext, List[AutoValueConvertingNeptuneRun]] = neptune_run_collected_results.context_indexed_runs_list_dict
# context_indexed_runs_list_dict

# most_recent_runs_table_df
context_indexed_run_logs: Dict[IdentifyingContext, str] = neptune_run_collected_results.context_indexed_run_logs
# context_indexed_run_logs

# run_logs: Dict[SessionDescriptorString, str] = neptune_run_collected_results.run_logs
# run_logs

## INPUTS: neptuner, run_logs, most_recent_runs_table_df

# session_column_individual_variables = ['format_name', 'animal', 'exper_name', 'session_name']
# session_column_variables = ['session_descriptor_string']
# processing_status_column_names = ['sys/id', 'sys/hostname', 'sys/creation_time', 'sys/running_time', 'sys/ping_time', 'sys/monitoring_time', 'sys/size', 'sys/tags', 'source_code/entrypoint']
# processing_status_column_names = ['sys/id', 'sys/hostname', 'sys/creation_time', 'sys/running_time', 'sys/ping_time', 'sys/monitoring_time', 'sys/size', 'sys/tags', 'source_code/entrypoint']
# processing_status_column_names = ['id', 'hostname', 'creation_time', 'running_time', 'ping_time', 'monitoring_time', 'size', 'tags', 'entrypoint']

most_recent_runs_table_df: pd.DataFrame = neptune_run_collected_results.most_recent_runs_table_df
most_recent_runs_table_df
most_recent_runs_session_descriptor_string_to_context_map: Dict[SessionDescriptorString, IdentifyingContext] = neptune_run_collected_results.most_recent_runs_session_descriptor_string_to_context_map
## INPUTS: most_recent_runs_session_descriptor_string_to_context_map, run_logs
context_indexed_run_logs: Dict[IdentifyingContext, str] = neptune_run_collected_results.context_indexed_run_logs # get the IdentifyingContext indexed item
# context_indexed_run_logs

# 'outputs/log'

## INPUTS: most_recent_runs_table_df
most_recent_runs_context_indexed_run_extra_data: Dict[IdentifyingContext, Dict] = neptune_run_collected_results.most_recent_runs_context_indexed_run_extra_data
# most_recent_runs_context_indexed_run_extra_data # SessionTuple(format_name='kdiba', animal='pin01', exper_name='one', session_name='11-02_17-46-44', session_descriptor_string='kdiba_pin01_one_11-02_17-46-44_sess', id='LS2023-1335', hostname='gl3126.arc-ts.umich.edu', creation_time=Timestamp('2024-08-29 16:39:16.613000'), running_time=8735.629, ping_time=Timestamp('2024-09-24 08:38:06.626000'), monitoring_time=1543, size=28686905.0, tags='11-02_17-46-44,one,kdiba,pin01', entrypoint='figures_kdiba_pin01_one_11-02_17-46-44.py')

# ['script_type']

## OUTPUTS: most_recent_runs_session_descriptor_string_to_context_map, context_indexed_run_logs, most_recent_runs_context_indexed_run_extra_data

In [8]:
# neptune_root_output_path = Path('EXTERNAL/PhoDibaPaper2024Book/data/neptune').resolve()
# neptune_root_output_path = Path(r"C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\PhoDibaPaper2024Book\data\neptune").resolve()
neptune_root_output_path = Path("/home/halechr/repos/Spike3D/EXTERNAL/PhoDibaPaper2024Book/data").resolve()
neptune_root_output_path.mkdir(exist_ok=True)
neptune_root_output_path
# neptune_project_output_path = neptune_root_output_path.joinpath(neptuner.project_name).resolve()
neptune_project_output_path = neptune_root_output_path.joinpath(neptuner.project_name).resolve()
neptune_project_output_path.mkdir(exist_ok=True, parents=True)
neptune_project_output_path

neptune_project_figures_output_path = neptune_project_output_path.joinpath('figs').resolve()
neptune_project_figures_output_path.mkdir(exist_ok=True, parents=True)
neptune_project_figures_output_path

neptune_project_figures_output_path = neptune_project_output_path.joinpath('figs').resolve()
neptune_project_figures_output_path.mkdir(exist_ok=True, parents=True)
neptune_project_figures_output_path

neptune_logs_output_path = neptune_project_output_path.joinpath('logs').resolve()
neptune_logs_output_path.mkdir(exist_ok=True, parents=True)
neptune_logs_output_path

PosixPath('/home/halechr/repos/Spike3D/data')

PosixPath('/home/halechr/repos/Spike3D/data/commander.pho/PhoDibaBatchProcessing')

PosixPath('/home/halechr/repos/Spike3D/data/commander.pho/PhoDibaBatchProcessing/figs')

PosixPath('/home/halechr/repos/Spike3D/data/commander.pho/PhoDibaBatchProcessing/figs')

PosixPath('/home/halechr/repos/Spike3D/data/commander.pho/PhoDibaBatchProcessing/logs')

In [11]:
_context_log_files_dict = neptune_run_collected_results.download_uploaded_log_files(neptune_logs_output_path=neptune_logs_output_path)
_context_log_files_dict

Fetching file...: 0 [00:00, ?/s]

Fetching file...: 0 [00:00, ?/s]

Fetching file...: 0 [00:00, ?/s]

MissingFieldException for a_run.id: PHDBATCH-1395


Fetching file...: 0 [00:00, ?/s]

...

Fetching file...: 0 [00:00, ?/s]

MissingFieldException for a_run.id: PHDBATCH-1381


Fetching file...: 0 [00:00, ?/s]

Fetching file...: 0 [00:00, ?/s]

{Context(format_name: 'kdiba'
         animal: 'vvp01'
         exper_name: 'one'
         session_name: '2006-4-10_12-25-50'): {'PHDBATCH-1380': '/home/halechr/repos/Spike3D/data/commander.pho/PhoDibaBatchProcessing/logs/kdiba|vvp01|one|2006-4-10_12-25-50|figures|PHDBATCH-1380.log',
  'PHDBATCH-1367': '/home/halechr/repos/Spike3D/data/commander.pho/PhoDibaBatchProcessing/logs/kdiba|vvp01|one|2006-4-10_12-25-50|run|PHDBATCH-1367.log',
  'PHDBATCH-1354': '/home/halechr/repos/Spike3D/data/commander.pho/PhoDibaBatchProcessing/logs/kdiba|vvp01|one|2006-4-10_12-25-50|run|PHDBATCH-1354.log'},
 Context(format_name: 'kdiba'
         animal: 'pin01'
         exper_name: 'one'
         session_name: '11-02_19-28-0'): {'PHDBATCH-1375': '/home/halechr/repos/Spike3D/data/commander.pho/PhoDibaBatchProcessing/logs/kdiba|pin01|one|11-02_19-28-0|run|PHDBATCH-1375.log',
  'PHDBATCH-1356': '/home/halechr/repos/Spike3D/data/commander.pho/PhoDibaBatchProcessing/logs/kdiba|pin01|one|11-02_19-28-0|run|PHDBAT

In [ ]:
def flatten_dict(d, parent_key='', sep='/'):
    if (not isinstance(d, dict)):
        assert isinstance(parent_key, str), f"expected type(parent_key) == str but instead type(parent_key): {type(parent_key)}, parent_key: {parent_key}"
        return {parent_key:d}
    
    items = {}
    for k, v in d.items():
        # Construct the new key by concatenating the parent key and current key
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            # If the value is a dictionary, recursively flatten it
            items.update(flatten_dict(v, new_key, sep=sep))
        else:
            # If the value is not a dictionary, add it to the items
            items[new_key] = v
    return items


context_indexed_runs_list_dict: Dict[IdentifyingContext, List[AutoValueConvertingNeptuneRun]] = neptune_run_collected_results.context_indexed_runs_list_dict
# context_indexed_runs_list_dict

_parsed_run_structure_dict = {}
_context_figures_dict = {}

for a_ctxt, a_run_list in context_indexed_runs_list_dict.items():
    _parsed_run_structure_dict[a_ctxt] = {}
    _context_figures_dict[a_ctxt] = {}
    for a_run in a_run_list:
        a_parsed_structure = a_run.get_structure()['outputs']['figures']
        assert isinstance(a_parsed_structure, dict), f"type(a_parsed_structure): {type(a_parsed_structure)} instead of dict. a_parsed_structure: {a_parsed_structure}"
        for k, v in a_parsed_structure.items():
            # Flatten each nested dictionary and update the flattened_dict
            _parsed_run_structure_dict[a_ctxt].update(flatten_dict(v, parent_key=k))

    
        # [{'display_fn_name:_display_grid_bin_bounds_validation': <neptune.attributes.atoms.file.File at 0x212bfbd4a30>,
        # 'display_fn_name:display_long_short_laps': <neptune.attributes.atoms.file.File at 0x212bfbd4a90>,
        # 'filter_name:maze1_any': {'lap_dir:any': {'display_fn_name:1d_placefields': <neptune.attributes.atoms.file.File at 0x212bfbd4bb0>,
        #     'display_fn_name:_display_1d_placefield_validations': <neptune.attributes.atoms.file.File at 0x212bfbd4c10>,
        #     'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': <neptune.attributes.atoms.file.File at 0x212bfbd4d30>,
        #     'display_fn_name:plot_occupancy': {'plot_variable:OCCUPANCY': <neptune.attributes.atoms.file.File at 0x212bfbd4d90>}}},
        # 'filter_name:maze1_even': {'lap_dir:even': {'display_fn_name:1d_placefields': <neptune.attributes.atoms.file.File at 0x212bfbd4f40>,
        #     'display_fn_name:_display_1d_placefield_validations': <neptune.attributes.atoms.file.File at 0x212bfbd4e80>,
        #     'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': <neptune.attributes.atoms.file.File at 0x212bf2e58e0>,
        #     'display_fn_name:plot_occupancy': {'plot_variable:OCCUPANCY': <neptune.attributes.atoms.file.File at 0x212bf2e5790>}}},
        # 'filter_name:maze1_odd': {'lap_dir:odd': {'display_fn_name:1d_placefields': <neptune.attributes.atoms.file.File at 0x212bf2e52b0>,
        #     'display_fn_name:_display_1d_placefield_validations': <neptune.attributes.atoms.file.File at 0x212bf2e5730>,
        #     'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': <neptune.attributes.atoms.file.File at 0x212b0cb1760>,
        #     'display_fn_name:plot_occupancy': {'plot_variable:OCCUPANCY': <neptune.attributes.atoms.file.File at 0x212b0cb1490>}}},
        # 'filter_name:maze2_any': {'lap_dir:any': {'display_fn_name:1d_placefields': <neptune.attributes.atoms.file.File at 0x212bf375a00>,
        #     'display_fn_name:_display_1d_placefield_validations': <neptune.attributes.atoms.file.File at 0x212bf37a190>,
        #     'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': <neptune.attributes.atoms.file.File at 0x212bf37afa0>,
        #     'display_fn_name:plot_occupancy': {'plot_variable:OCCUPANCY': <neptune.attributes.atoms.file.File at 0x212bf37d130>}}},
        # 'filter_name:maze2_even': {'lap_dir:even': {'display_fn_name:1d_placefields': <neptune.attributes.atoms.file.File at 0x212d8381700>,
        #     'display_fn_name:_display_1d_placefield_validations': <neptune.attributes.atoms.file.File at 0x212bfbd3760>,
        #     'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': <neptune.attributes.atoms.file.File at 0x212bfbd3610>,
        #     'display_fn_name:plot_occupancy': {'plot_variable:OCCUPANCY': <neptune.attributes.atoms.file.File at 0x212bfbd35b0>}}},
        # 'filter_name:maze2_odd': {'lap_dir:odd': {'display_fn_name:1d_placefields': <neptune.attributes.atoms.file.File at 0x212bfbd3340>,
        #     'display_fn_name:_display_1d_placefield_validations': <neptune.attributes.atoms.file.File at 0x212bfbd34c0>,
        #     'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': <neptune.attributes.atoms.file.File at 0x212bfbd31f0>,
        #     'display_fn_name:plot_occupancy': {'plot_variable:OCCUPANCY': <neptune.attributes.atoms.file.File at 0x212bfbd3190>}}},
        # 'filter_name:maze_any': {'lap_dir:any': {'display_fn_name:1d_placefields': <neptune.attributes.atoms.file.File at 0x212bfbd38b0>,
        #     'display_fn_name:_display_1d_placefield_validations': <neptune.attributes.atoms.file.File at 0x212bfbd3130>,
        #     'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': <neptune.attributes.atoms.file.File at 0x212bfbd3b50>,
        #     'display_fn_name:plot_occupancy': {'plot_variable:OCCUPANCY': <neptune.attributes.atoms.file.File at 0x212bfbd3ca0>}}},
        # 'filter_name:maze_even': {'lap_dir:even': {'display_fn_name:1d_placefields': <neptune.attributes.atoms.file.File at 0x212bfbd3970>,
        #     'display_fn_name:_display_1d_placefield_validations': <neptune.attributes.atoms.file.File at 0x212bfbd3be0>,
        #     'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': <neptune.attributes.atoms.file.File at 0x212c0df0c40>,
        #     'display_fn_name:plot_occupancy': {'plot_variable:OCCUPANCY': <neptune.attributes.atoms.file.File at 0x212c0df0cd0>}}},
        # 'filter_name:maze_odd': {'lap_dir:odd': {'display_fn_name:1d_placefields': <neptune.attributes.atoms.file.File at 0x212c0df0eb0>,
        #     'display_fn_name:_display_1d_placefield_validations': <neptune.attributes.atoms.file.File at 0x212c0df0d90>,
        #     'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': <neptune.attributes.atoms.file.File at 0x212bfbc3580>,
        #     'display_fn_name:plot_occupancy': {'plot_variable:OCCUPANCY': <neptune.attributes.atoms.file.File at 0x212bfbc33d0>}}}},
        #     ],
        
        # {'display_fn_name:_display_grid_bin_bounds_validation': <neptune.attributes.atoms.file.File at 0x212bfbd4a30>,
        # 'display_fn_name:display_long_short_laps': <neptune.attributes.atoms.file.File at 0x212bfbd4a90>,
        # 'filter_name:maze1_any,lap_dir:any,display_fn_name:1d_placefields': <neptune.attributes.atoms.file.File at 0x212bfbd4bb0>,
        # 'filter_name:maze1_any,lap_dir:any,display_fn_name:_display_1d_placefield_validations': <neptune.attributes.atoms.file.File at 0x212bfbd4c10>,
        # 'filter_name:maze1_any,lap_dir:any,display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': <neptune.attributes.atoms.file.File at 0x212bfbd4d30>,
        # 'filter_name:maze1_any,lap_dir:any,display_fn_name:plot_occupancy,plot_variable:OCCUPANCY': <neptune.attributes.atoms.file.File at 0x212bfbd4d90>,
        # ...
        # }
    # END FOR a_run
    ## get the values for each File
    # for k, v in _parsed_run_structure_dict[a_ctxt].items():
    #     _context_figures_dict[a_ctxt][k] = v.download()
        # _context_figures_dict[a_ctxt][k] = v.download(destination=neptune_project_figures_output_path.as_posix())


_parsed_run_structure_dict
# _context_figures_dict

In [ ]:
most_recent_runs_table_df.to_csv('output/2024-09-24_most_recent_neptune_runs_csv.csv')

In [ ]:
most_recent_runs_table_df['final_log_file_path']


In [ ]:
## INPUTS: context_indexed_run_logs, neptune_logs_output_path
## OUTPUT: _out_log_paths
neptune_logs_output_path = neptune_project_output_path.joinpath('logs').resolve()
neptune_logs_output_path.mkdir(exist_ok=True, parents=True)
_out_log_paths = neptune_run_collected_results._perform_export_log_files_to_separate_files(context_indexed_run_logs=context_indexed_run_logs, neptune_logs_output_path=neptune_logs_output_path)
_out_log_paths

# Trivial ipytree Example that causes jumping

In [ ]:
from IPython.display import clear_output, display
from ipytree import Tree, Node


tree = Tree(stripes=False)
for i in range(3):
    node = Node(f"Node {i}")
    tree.add_node(node)
    
def handle_click(event):
    clear_output(wait=True)  # Prevent scrolling
    display(tree)  # Re-display the tree to keep focus
    print(f"Clicked on: {event['owner'].name}")

for node in tree.nodes:
    node.observe(handle_click, names='selected')

tree


# Log Viewer Widget

In [ ]:
# Call the function to build and display the interactive session display
interactive_layout = Neptuner.build_interactive_session_run_logs_widget(context_indexed_run_logs, most_recent_runs_session_descriptor_string_to_context_map, most_recent_runs_context_indexed_run_extra_data)
display(interactive_layout)

In [ ]:
# does not trigger change:
# .on_node_selected(change: {'name': 'selected', 'old': True, 'new': False, 'owner': Node(name='11-02_17-46-44'), 'type': 'change'})
# .on_node_selected(change: {'name': 'selected', 'old': False, 'new': True, 'owner': Node(name='fet11-01_12-58-54', selected=True), 'type': 'change'})
# Selected context: {'format_name': 'kdiba', 'animal': 'pin01', 'exper_name': 'one', 'session_name': 'fet11-01_12-58-54'}


In [ ]:
import ipywidgets as widgets
from IPython.display import display
import json

def save_widget_as_html(widget, filename):
    """Saves the widget as a self-contained HTML file."""
    from ipywidgets.embed import embed_minimal_html
    
    embed_minimal_html(filename, views=[widget], title="Interactive Session Display")

# Assuming 'root_box' is your widget
export_html_path = Path(r'C:\Users\pho\repos\Spike3DWorkEnv\Spike3D\EXTERNAL\PhoDibaPaper2024Book\data\neptune').joinpath('interactive_session_display.html').resolve()
save_widget_as_html(interactive_layout, export_html_path)
export_html_path


In [ ]:
empty_session_tuple = {
    'format_name': '',
    'animal': '',
    'exper_name': '',
    'session_name': '',
    'session_descriptor_string': '',
    'id': '<Selection Not Found>',
    'hostname': '',
    'creation_time': '',
    'running_time': '',
    'ping_time': '',
    'monitoring_time': '',
    'size': '',
    'tags': '',
    'entrypoint': ''
}

print(list(empty_session_tuple.keys())) # ['format_name', 'animal', 'exper_name', 'session_name', 'session_descriptor_string', 'id', 'hostname', 'creation_time', 'running_time', 'ping_time', 'monitoring_time', 'size', 'tags', 'entrypoint']
display_session_extra_data_keys = ['id', 'hostname', 'creation_time', 'running_time', 'ping_time', 'monitoring_time', 'size', 'tags', 'entrypoint']

# hostname: gl3126.arc-ts.umich.edu
# creation_time: 2024-08-29 16:47:55.150000
# running_time: 8512.876
# ping_time: 2024-09-24 08:38:06.349000
# monitoring_time: 1291
# size: 24744223.0
# tags: one,kdiba,11-02_19-28-0,pin01
# entrypoint: figures_kdiba_pin01_one_11-02_19-28-0.py



In [ ]:
from pandas import Timestamp
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.gui.Jupyter.TreeWidget import JupyterTreeWidget

## INPUTS: context_indexed_run_logs, most_recent_runs_session_descriptor_string_to_context_map, header_hbox

# Tree Widget ________________________________________________________________________________________________________ #
included_session_contexts: List[IdentifyingContext] = list(most_recent_runs_session_descriptor_string_to_context_map.values())
jupyter_tree_widget = JupyterTreeWidget(included_session_contexts=included_session_contexts,
										on_selection_changed_callbacks=[],
										display_on_init=False)
# type(jupyter_tree_widget.tree) # ipytree.tree.Tree

# Set a layout for the tree to prevent it from being cut off
jupyter_tree_widget.tree.layout = widgets.Layout(min_width='200px', max_width='600px', overflow='auto', height='auto')

# Content Widget _____________________________________________________________________________________________________ #
def build_session_tuple_header_widget(a_session_tuple: Tuple):
    # Create widgets for each key-value pair
    # widgets_list = [widgets.Label(f"{key}: {value}") for key, value in a_session_tuple.items()]
    
    # Create a dictionary to hold the label widgets
    header_label_widgets = {key: widgets.Label(f"{key}: '{value}',") for key, value in a_session_tuple.items()}

    # Horizontally stack the label widgets
    # header_hbox = widgets.HBox(list(header_label_widgets.values()), layout=widgets.Layout(min_width='400px', min_height='50px', width='auto', height='auto')) # , overflow='auto'
    
    # Define a layout that enables wrapping
    box_layout = widgets.Layout(
        display='flex',
        flex_flow='row wrap',
        align_items='stretch',
        width='100%'
    )

    # Create a Box with the custom layout
    header_hbox = widgets.Box(list(header_label_widgets.values()), layout=box_layout)


    # Function to update the values in the labels
    def update_header_labels_fn(new_values):
        """ captures: label_widgets"""
        for key, value in new_values.items():
            header_label_widgets[key].value = f"{key}: {value}"
            
    # Display the widget
    return header_hbox, header_label_widgets, update_header_labels_fn


# Example SessionTuple
empty_session_tuple = {
    'format_name': '',
    'animal': '',
    'exper_name': '',
    'session_name': '',
    'session_descriptor_string': '',
    'id': '<Selection Not Found>',
    'hostname': '',
    'creation_time': '',
    'running_time': '',
    'ping_time': '',
    'monitoring_time': '',
    'size': '',
    'tags': '',
    'entrypoint': ''
}

header_hbox, header_label_widgets, update_header_labels_fn = build_session_tuple_header_widget(a_session_tuple=empty_session_tuple)

# Create Textarea widget with a defined width
textarea = widgets.Textarea(value='<No Selection>',
    disabled=True,  # Make it read-only
    style={'font_size': '10px'},  # Smaller font size
	layout=widgets.Layout(flex='1', width='650px', min_height='650px',
                        height='850px',
                        ))

content_view_layout = widgets.VBox([header_hbox, textarea], layout=widgets.Layout(min_width='500px', min_height='200px', width='auto', height='auto')) # INPUTS: header_hbox

# Layout widgets side by side with proper spacing
layout = widgets.HBox([jupyter_tree_widget.tree, content_view_layout], layout=widgets.Layout(min_width='500px', min_height='100px', width='auto', height='auto'))


def _on_tree_node_selection_changed(selected_node, selected_context):
    """ 
    captures: context_indexed_run_logs, textarea, most_recent_runs_context_indexed_run_extra_data, empty_session_tuple, update_header_labels_fn
    """
    print(f'_on_tree_node_selection_changed(selected_node: {selected_node}, selected_context: {selected_context})') # Selected context: {'format_name': 'kdiba', 'animal': 'pin01', 'exper_name': 'one', 'session_name': 'fet11-01_12-58-54'}
    #_on_tree_node_selection_changed(selected_node: Node(name='fet11-01_12-58-54', selected=True), selected_context: {'format_name': 'kdiba', 'animal': 'pin01', 'exper_name': 'one', 'session_name': 'fet11-01_12-58-54'})
    if isinstance(selected_context, dict):
        selected_context = IdentifyingContext(**selected_context)
    
    curr_context_extra_data_tuple = most_recent_runs_context_indexed_run_extra_data.get(selected_context, empty_session_tuple)
    update_header_labels_fn(curr_context_extra_data_tuple)
    curr_context_run_log: str = context_indexed_run_logs.get(selected_context, '<Context Not Found>')
    textarea.value = curr_context_run_log

jupyter_tree_widget.on_selection_changed_callback = [_on_tree_node_selection_changed]
# Display the layout
layout


In [ ]:


text_area = widgets.Textarea(
    value='<No Selection>',
    disabled=True,  # Make it read-only
    layout=widgets.Layout(width='100%', height='400px')  # Scrollable area
)

combined_widget = widgets.HBox([jupyter_tree_widget.tree, text_area])
combined_widget
# log_viewer = create_log_viewer(logs=run_logs)
# display(log_viewer)


In [ ]:
parsed_structure = run.get_parsed_structure()
parsed_structure

In [ ]:
import regions
(parsed_structure['monitoring'])


In [ ]:
for run_id, a_log in run_logs.items():
    print("# ==================================================================================================================== #")
    print(f"# run_id: {run_id}                                                                                                     #")
    print("# ==================================================================================================================== #")    
    # print(f'run_id: {run_id} =================')
    # print(f"# run_id: {run_id:<100} #")
    print(a_log)
    print("# END LOG ____________________________________________________________________________________________________________ #")
    


Calling `run.print_structure()` produces the following output:
```python
'animal': String
'exper_name': String
'format_name': String
'monitoring':
    '5f739afe':
        'hostname': String
        'pid': String
        'tid': String
    'be28f54f':
        'cpu': FloatSeries
        'hostname': String
        'memory': FloatSeries
        'pid': String
        'stderr': StringSeries
        'stdout': StringSeries
        'tid': String
'outputs':
    'figures':
        'display_fn_name:BatchPhoJonathanReplayFRC':
            'plot_result_set:long_only':
                'aclus:(16,19,6,31,10)': File
                'aclus:(6,10,16,19,31)': File
            'plot_result_set:shared':
                'page:1of2':
                    'aclus:(26,20,29,18,3,4,7,32,13,11,12,2,30,15,9,28,22,14,21,25)': File
                'page:1of3':
                    'aclus:(2,3,4,5,7,8,9,11,12,13)': File
                'page:2of2':
                    'aclus:(5,8,17,23,27)': File
                'page:2of3':
                    'aclus:(14,15,17,18,20,21,22,23,25,26)': File
                'page:3of3':
                    'aclus:(27,28,29,30,32)': File
            'plot_result_set:short_only':
                'aclus:(24)': File
        'display_fn_name:_display_grid_bin_bounds_validation': File
        'display_fn_name:bidir_track_remap':
            'subplot_name:Track Remapping': File
        'display_fn_name:display_long_short_laps': File
        'display_fn_name:display_short_long_pf1D_comparison':
            'track:long': File
            'track:short': File
        'display_fn_name:plot_all_epoch_bins_marginal_predictions':
            'subplot_name:Laps all_epoch_binned Marginals': File
            'subplot_name:Ripple all_epoch_binned Marginals': File
        'display_fn_name:plot_expected_vs_observed':
            'x_variable:epoch_idx':
                'variable:obs_exp_diff_ptp': File
        'display_fn_name:plot_histograms':
            'subplot_name:laps_result_tuple': File
            'subplot_name:ripple_result_tuple': File
        'display_fn_name:plot_quantile_diffs':
            'subplot_name:BestDir': File
        'display_fn_name:plot_rank_order_epoch_inst_fr_result_tuples':
            'subplot_name:Lap':
                'subplotsubplot_name:raw': File
                'subplotsubplot_name:z_score_diff': File
            'subplot_name:Ripple':
                'subplotsubplot_name:raw': File
                'subplotsubplot_name:z_score_diff': File
        'display_fn_name:plot_rank_order_histograms':
            'subplot_name:Ripple Most-likely Spearman Rho': File
            'subplot_name:Ripple Most-likely Z-scores': File
            'subplot_name:Ripple Z-scores': File
            'subplot_name:Ripple real correlations': File
        'display_fn_name:running_and_replay_speeds_over_time': File
        'filter_name:maze1_any':
            'lap_dir:any':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
                'display_fn_name:plot_single_track_firing_rate_compare': File
        'filter_name:maze1_even':
            'lap_dir:even':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze1_odd':
            'lap_dir:odd':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze2_any':
            'lap_dir:any':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
                'display_fn_name:plot_single_track_firing_rate_compare': File
        'filter_name:maze2_even':
            'lap_dir:even':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze2_odd':
            'lap_dir:odd':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze_any':
            'lap_dir:any':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze_even':
            'lap_dir:even':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'filter_name:maze_odd':
            'lap_dir:odd':
                'display_fn_name:1d_placefields': File
                'display_fn_name:_display_1d_placefield_validations': File
                'display_fn_name:_display_2d_placefield_result_plot_ratemaps_2D': File
                'display_fn_name:plot_occupancy':
                    'plot_variable:OCCUPANCY': File
        'fn_name:long_short_firing_rate_indicies':
            'display_fn_name:display_long_short_laps': File
'session_descriptor_string': String
'session_name': String
'source_code':
    'diff': File
    'entrypoint': String
'sys':
    'creation_time': Datetime
    'description': String
    'failed': Boolean
    'group_tags': StringSet
    'hostname': String
    'id': String
    'modification_time': Datetime
    'monitoring_time': Integer
    'name': String
    'owner': String
    'ping_time': Datetime
    'running_time': Float
    'size': Float
    'state': RunState
    'tags': StringSet
    'trashed': Boolean
```
How can I capture this printed output (printed to stdout I think) and then parse it into a tree-like structure made of nested-dictionaries to be able to figure out the available structure?

In [ ]:
# Fetch all experiments
experiments = neptuner.project.fetch_experiments_table().to_pandas()
experiments

In [ ]:
# Initialize the neptune client
# neptune.init(project=neptune_kwargs['project'], api_token=neptune_kwargs['api_token'])

# project = neptune.get_project()
# project

In [ ]:


if neptuner.run is None:
    neptuner.run = AutoValueConvertingNeptuneRun(project=neptuner.project_name, api_token=neptuner.api_token, dependencies="infer", source_files=[notebook_filepath])
    params = {"TODAY_DAY_DATE": TODAY_DAY_DATE, "run_workstation": "Apogee"}
    neptuner.run["parameters"] = params
    neptuner.outputs = neptuner.run['outputs']
    neptuner.figures = neptuner.outputs['figures']

neptuner_run: AutoValueConvertingNeptuneRun = neptuner.run

# run = neptune.init_run(source_files=["**/*.dvc"])

# # Pre-execution dataframe view:
# run["dataset/global_batch_run_progress_df"].upload(File.as_html(global_batch_run.to_dataframe(expand_context=True, good_only=False))) # "path/to/test_preds.csv"



known_bad_sessions = [IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44')]
known_bad_session_strs = [str(v.get_description()) for v in known_bad_sessions]
known_bad_session_strs